In [9]:
!pip install --upgrade openai

In [1]:
# open train.csv
# for each row, concatenate columns 'prompt' to 'E' and add the text 
# save it to a new file
import csv 

with open('train.csv', 'r') as csv_file:
    # Create a CSV reader
    reader = csv.reader(csv_file)
    
    # Read the header
    header = next(reader)
    
    # Find the indices for columns B to G
    b_index, g_index = header.index('prompt'), header.index('E')

    # Read all rows into a list
    rows = [row for row in reader]

# Process the rows
appended_rows = []
ids = []
labels = []
for row in rows:
    # Concatenate cells B to G with a preceding A, B, C, D, or E 
    concat_text = row[b_index] + ' A. ' + row[b_index+1] + ' B. ' + row[b_index+2] + ' C. ' + row[b_index+3] + ' D. ' + row[b_index+4] + ' E. ' + row[b_index+5]
    ids.append(row[0])
    labels.append(row[7])
    text = f"Answer the following question with A, B, C, D, or E. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. {concat_text}"
    appended_rows.append(text)

# Write the result to the output CSV file
with open('apiGPT.csv', 'w', newline='') as csv_file:
    # Create a CSV writer
    writer = csv.writer(csv_file)
    # put ids and appended rows into a list
    prompts = [[i, j] for i, j in zip(ids, appended_rows)]
    # Write the header  with the ids and the appended rows to the output file
    writer.writerow(['id', 'text'])
    writer.writerows(prompts)

In [4]:
from openai import OpenAI

client = OpenAI(
  api_key='TOKEN',
)


SyncPage[Model](data=[Model(id='text-search-babbage-doc-001', created=1651172509, object='model', owned_by='openai-dev'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='curie-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='text-davinci-003', created=1669599635, object='model', owned_by='openai-internal'), Model(id='text-search-babbage-query-001', created=1651172509, object='model', owned_by='openai-dev'), Model(id='babbage', created=1649358449, object='model', owned_by='openai'), Model(id='babbage-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='text-babbage-001', created=1649364043, object='model', owned_by='openai'), Model(id='text-similarity-davinci-001', created=1651172505, object='model', owned_by='openai-dev'), Model(id='davinci-similarity', created=1651172509, object='model', o

In [5]:
# call the api on each prompt and save the result to a new file
answers = []
for prompt in prompts:
    answer =client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt= prompt[1], 
    )
    if answer.choices[0].text[2] in ['A', 'B', 'C', 'D', 'E']:
        answers.append(answer.choices[0].text[2])
    else:
        answers.append(answer.choices[0].text)

In [6]:
# write the answers to the existing apiGPT.csv with a new column called 'predicted'
import pandas as pd

file_path = 'apiGPT.csv'
df = pd.read_csv(file_path)
df['predicted'] = answers
df.to_csv(file_path, index=False)

# go to the csv and manually fix the predictions that need cleaning

In [7]:
file_path = 'apiGPT.csv'
df = pd.read_csv(file_path)

In [9]:
# compare the answers to the correct answers and calculate the accuracy
correct = 0
incorrect = 0

for label, answer in zip(labels, df['predicted']):
    print(label, answer)
    if label == answer.strip():
        correct += 1
    else:
        incorrect += 1

    

D B
A A
A B
C A
D D
B B
A E
D B
C B
A A
E D
A A
C E
D E
B B
B D
E A
E E
A E
E E
D D
D D
C C
C C
E E
E E
A A
D B
E B
C C
B D
E E
E A
D D
C E
B B
E A
A A
E E
E This question does not make sense as there is no such thing as a syn
E E
C D
B B
C C
A A
A A
B B
C C
D A
B B
B B
E E
C B
A B
B D
B B
C C
C B
D D
A A
B B
B A
C C
C C
A A
E D
C A
E E
C A
D D
C C
A A
D D
B E
D D
B B
D D
B B
C C
E E
C C
A A
B B
A A
C C
D D
D A
B C
E E
D A
B B
B B
B B
E This question does not make any sense. Supersymmetry is a concept
E E
C C
C C
D A
D D
D A
D D
B A
C C
B C
C C
D E
A A
D E
A A
D C
C C
D D
A This question does not make sense as excess quarks are not needed to provide
B B
D D
E E
B B
C A
D D
A A
A A
C C
B B
A D
B A
D D
B B
C C
E E
E E
A A
C A
E B
B B
C C
E E
E A
D B
A A
A A
C C
E A
B B
A A
C C
B B
B B
C A
B B
B B
B B
D D
A E
A A
B E
E E
D E
C A
A A
A A
A A
E B
B A
B B
C E
D A
B A
D C
E E
A A
B D
E B
B B
C C
B B
E A
D E
D D
C C
C E
B B
A A
A A
C C
A A
A A
C C
A This question does not make sense as there i

In [10]:
correct
# incorrect 

125

In [11]:
incorrect

75